In [1]:
import random
import itertools
import threading
import queue
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
import pandas as pd
import speaker_buddy as buddy

### Exchanges

In [2]:
EXCHANGES= [
    'bitfinex',
    'coinbase-pro',
    'bitstamp',
    'kraken',
    'cexio',
    'okcoin',
    'bitmex',
    'mexbt',
    'huobi',
    'poloniex',
    'bittrex',
    'okex',
    'hitbtc'
]

### Symbols

In [3]:
SYMBOLS = ['btc', 'ltc', 'eth']

### Combinations

In [4]:
ALL_COMBINATIONS = list(itertools.product(EXCHANGES, SYMBOLS))

### Random Combinatios

In [5]:
combinations = [random.choice(ALL_COMBINATIONS) for _ in range(10)]

### Random Dates

In [6]:
ALL_DATES = [d.strftime('%Y-%m-%d') for d in pd.date_range('2019-01-01', '2019-07-20', freq='D')]

In [7]:
dates = [random.choice(ALL_DATES) for _ in range(5)]

### Summary

In [8]:
dates

['2019-06-27', '2019-06-02', '2019-03-21', '2019-03-14', '2019-03-09']

In [9]:
combinations

[('bitstamp', 'eth'),
 ('bitmex', 'btc'),
 ('coinbase-pro', 'ltc'),
 ('bitmex', 'ltc'),
 ('bittrex', 'btc'),
 ('bitmex', 'ltc'),
 ('cexio', 'ltc'),
 ('huobi', 'eth'),
 ('okex', 'eth'),
 ('cexio', 'ltc')]

In [10]:
data = list(itertools.product(combinations, dates))
data[:3]

[(('bitstamp', 'eth'), '2019-06-27'),
 (('bitstamp', 'eth'), '2019-06-02'),
 (('bitstamp', 'eth'), '2019-03-21')]

In [11]:
URL = 'http://localhost:5000/price/{exchange}/{symbol}/{date}'

In [12]:
price_urls = [
        URL.format(exchange=exchange, symbol=symbol, date=date)
    for (exchange, symbol), date in data]

In [13]:
price_urls[:3]

['http://localhost:5000/price/bitstamp/eth/2019-06-27',
 'http://localhost:5000/price/bitstamp/eth/2019-06-02',
 'http://localhost:5000/price/bitstamp/eth/2019-03-21']

In [14]:
len(price_urls)

50

### A simple test

In [15]:
resp = requests.get(price_urls[0])

In [16]:
resp.json()

{'exchange': 'bitstamp',
 'symbol': 'eth',
 'open': 316.26,
 'high': 363.18,
 'low': 310,
 'close': 334.78,
 'volume': 159911.94085667,
 'day': '2019-06-27'}

We'll do the same thing for all of them.

---

# Multi threaded

In [17]:
THREAD_COUNT = 5

In [18]:
import queue

In [19]:
url_queue = queue.Queue()
results_queue = queue.Queue()
for url in price_urls:
    url_queue.put(url)

In [20]:
import time
class PriceProcessThread(threading.Thread):
    def __init__(self, url_queue, results_queue, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.url_queue = url_queue
        self.results_queue = results_queue
        
    def run(self):
        while True:
            url = self.url_queue.get()
            resp = requests.get(url)
            self.results_queue.put((url, resp.json()))
            self.url_queue.task_done()

In [21]:
for _ in range(THREAD_COUNT):
    worker = PriceProcessThread(url_queue, results_queue, daemon=True)
    worker.start()

In [22]:
url_queue.join()

In [23]:
url_queue.qsize()

0

In [24]:
results_queue.qsize()

50

In [25]:
EXCHANGES = {}

In [26]:
while not results_queue.empty():
    url, result = results_queue.get()
    (*_, exchange, symbol, date) = url.split('/')
    print(f"{date} - {exchange} - {symbol.upper()}: {(result or {}).get('close')}")

2019-06-27 - bitstamp - ETH: 334.78
2019-06-02 - bitstamp - ETH: 264.56
2019-03-14 - bitstamp - ETH: 131.16
2019-03-09 - bitstamp - ETH: 132.8
2019-03-21 - bitstamp - ETH: 138.71
2019-06-27 - bitmex - BTC: None
2019-06-02 - bitmex - BTC: None
2019-03-21 - bitmex - BTC: None
2019-03-09 - bitmex - BTC: None
2019-03-14 - bitmex - BTC: None
2019-06-27 - coinbase-pro - LTC: 130.64
2019-06-02 - coinbase-pro - LTC: 112.48
2019-03-21 - coinbase-pro - LTC: 60.21
2019-03-09 - coinbase-pro - LTC: 55.33
2019-03-14 - coinbase-pro - LTC: 55.06
2019-06-27 - bitmex - LTC: None
2019-06-02 - bitmex - LTC: None
2019-03-21 - bitmex - LTC: None
2019-03-14 - bitmex - LTC: None
2019-03-09 - bitmex - LTC: None
2019-06-27 - bittrex - BTC: 12763.001
2019-06-02 - bittrex - BTC: 8555.879
2019-03-21 - bittrex - BTC: 4036.183
2019-03-14 - bittrex - BTC: 3850.675
2019-03-09 - bittrex - BTC: 3842.137
2019-06-02 - bitmex - LTC: None
2019-06-27 - bitmex - LTC: None
2019-03-21 - bitmex - LTC: None
2019-03-14 - bitmex - 

## `concurrent.futures`

The code we have produced includes multiple of the issues we mentioned before. `concurrent.futures` is a library that includes a high level abstraction for concurrency, with many issues already solved. The library is very well built and its simplicity relies in 2 main abstractions:

* The `Executor`: in charge of scheduling and synchronizing jobs
* A `Future`: an object encapsulating the state of a concurrent computation

In [27]:
import time

In [28]:
from concurrent.futures import ThreadPoolExecutor

With the `Executor` we'll schedule multiple tasks, we'll define the task as a simple function:

In [74]:
def get_price(url):
    time.sleep(.25)
    resp = requests.get(url)
    return resp.json()

We want to run this for each URL:

In [30]:
get_price(price_urls[0])

{'exchange': 'bitstamp',
 'symbol': 'eth',
 'open': 316.26,
 'high': 363.18,
 'low': 310,
 'close': 334.78,
 'volume': 159911.94085667,
 'day': '2019-06-27'}

Now the code. First we create an executor:

In [75]:
executor = ThreadPoolExecutor(max_workers=THREAD_COUNT)

We "schedule" all the jobs with the `submit()` method. Each `submit()` call returns a `Future` (that we'll use later).

In [76]:
futures = [executor.submit(get_price, url) for url in price_urls]

In [77]:
futures

[<Future at 0x1133df450 state=finished returned dict>,
 <Future at 0x1133dffd0 state=finished returned dict>,
 <Future at 0x1133dfe50 state=finished returned dict>,
 <Future at 0x1133ec810 state=finished returned dict>,
 <Future at 0x1133ece90 state=finished returned dict>,
 <Future at 0x1133eca10 state=running>,
 <Future at 0x1133ecd10 state=running>,
 <Future at 0x1133ec790 state=running>,
 <Future at 0x1133ecbd0 state=running>,
 <Future at 0x1133ec8d0 state=running>,
 <Future at 0x1133ec910 state=pending>,
 <Future at 0x1133ec5d0 state=pending>,
 <Future at 0x1133ec850 state=pending>,
 <Future at 0x1133ec510 state=pending>,
 <Future at 0x1133eca90 state=pending>,
 <Future at 0x1133ec250 state=pending>,
 <Future at 0x1133ec210 state=pending>,
 <Future at 0x1133ec090 state=pending>,
 <Future at 0x1133dd550 state=pending>,
 <Future at 0x1134a66d0 state=pending>,
 <Future at 0x1134a6a90 state=pending>,
 <Future at 0x1134a61d0 state=pending>,
 <Future at 0x1134a6cd0 state=pending>,
 <Fut

Most of the `Future`s will be _pending_. They're still being executed by the `Executor`.

In [78]:
f = futures[-1]

In [81]:
f.done()

True

In [82]:
f.running()

False

In [73]:
f.result()  # will block until a result arrives

{'exchange': 'cexio',
 'symbol': 'ltc',
 'open': 57.05,
 'high': 58.78,
 'low': 55.44,
 'close': 57.04,
 'volume': 865.43857064,
 'day': '2019-03-09'}

We can use the `as_completed` module-level function to produce start receiving results as they're completed:

In [84]:
for future in as_completed(futures):
    print(future.result())

{'exchange': 'cexio', 'symbol': 'ltc', 'open': 137.38, 'high': 139, 'low': 126.04, 'close': 132.32, 'volume': 2025.74943904, 'day': '2019-06-27'}
{'exchange': 'huobi', 'symbol': 'eth', 'open': 267.97, 'high': 275.25, 'low': 260.88, 'close': 264.28, 'volume': 87412.29847513558, 'day': '2019-06-02'}
{'exchange': 'cexio', 'symbol': 'ltc', 'open': 57.05, 'high': 58.78, 'low': 55.44, 'close': 57.04, 'volume': 865.43857064, 'day': '2019-03-09'}
{'exchange': 'bitstamp', 'symbol': 'eth', 'open': 268.37, 'high': 274.86, 'low': 260.88, 'close': 264.56, 'volume': 33250.87175678, 'day': '2019-06-02'}
{'exchange': 'cexio', 'symbol': 'ltc', 'open': 115.18, 'high': 116.76, 'low': 112.12, 'close': 112.3, 'volume': 393.44924916, 'day': '2019-06-02'}
None
{'exchange': 'bittrex', 'symbol': 'btc', 'open': 3860.3190000000004, 'high': 3910, 'low': 3773.8059999999996, 'close': 3842.137, 'volume': 676.23358273, 'day': '2019-03-09'}
None
{'exchange': 'bitstamp', 'symbol': 'eth', 'open': 136.45, 'high': 138.59,

**Important!** We must shutdown the Executor once we're done using it. This way it'll free the resources:

In [85]:
executor.shutdown()

### A better approach

The `Executor` class can act as a Context Manager (`with`). That allows us to simplify the above code in:

In [86]:
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(get_price, url) for url in price_urls]

    for future in as_completed(futures):
        print(future.result())

{'exchange': 'bitstamp', 'symbol': 'eth', 'open': 316.26, 'high': 363.18, 'low': 310, 'close': 334.78, 'volume': 159911.94085667, 'day': '2019-06-27'}
{'exchange': 'bitstamp', 'symbol': 'eth', 'open': 268.37, 'high': 274.86, 'low': 260.88, 'close': 264.56, 'volume': 33250.87175678, 'day': '2019-06-02'}
None
{'exchange': 'coinbase-pro', 'symbol': 'ltc', 'open': 59.27, 'high': 60.91, 'low': 56.8, 'close': 60.21, 'volume': 155413.65331532, 'day': '2019-03-21'}
{'exchange': 'coinbase-pro', 'symbol': 'ltc', 'open': 114.45, 'high': 116.11, 'low': 111, 'close': 112.48, 'volume': 213514.40823597, 'day': '2019-06-02'}
None
None
{'exchange': 'bitstamp', 'symbol': 'eth', 'open': 137.87, 'high': 139.31, 'low': 135.26, 'close': 138.71, 'volume': 22614.42793805, 'day': '2019-03-21'}
None
None
{'exchange': 'bitstamp', 'symbol': 'eth', 'open': 136.45, 'high': 138.59, 'low': 128.85, 'close': 132.8, 'volume': 48116.71528489, 'day': '2019-03-09'}
None
None
None
{'exchange': 'coinbase-pro', 'symbol': 'ltc